In [6]:
from konlpy.tag import Okt
from konlpy.tag import Kkma
from kiwipiepy import Kiwi
import pandas as pd
import numpy as np
import koreanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from PIL import Image 
import re
from collections import Counter
from emoji import core

In [16]:
df = pd.read_csv(
    "../data/saramin_merge_sections_parsed_overwrite_v6.csv"
)

df = df.drop(columns=["Unnamed: 0"])

df.head()

,keyword,title,company,location,career,education,work_type,deadline,link,rec_idx,...,자격요건,우대사항,채용절차,복지 및 혜택,급여,parse_method,parse_confidence,needs_review,review_notes,sectioned_text
0,AI 에이전트,[AI기반 특허 데이터분석 플랫폼 기업] 프론트엔드 개발자,에이피써치,서울 강남구,경력 7~15년,대졸↑,정규직,채용시,https://www.saramin.co.kr/zf_user/jobs/relay/v...,52982351,...,] \n- Frontend 개발 경험을 7년 이상 보유하고 있으신 분\n- Java...,]\n- TypeScript에 대한 경험과 이해가 깊은 분\n- 대규모 프로젝트나 ...,접수기간 : 2026-02-02 11시 ~ 채용시\n제출서류 : 사람인 온라인 이력...,NaN,(연봉) : 추후협의\n [제출서류]\n- 국문 이력서 및 경력기술서 (Word로 ...,header+rule,0.794,False,moved_process_lines,주요업무:\n[회사]\n- AI 기반 특허 데이터분석 플랫폼 스타트업\n- 2025...
1,AI 에이전트,[자율주행] AI Engineer / 3D Auto-labeling,(주)비저너리,서울 강남구,경력무관,석사↑,정규직,채용시,https://www.saramin.co.kr/zf_user/jobs/relay/v...,52981267,...,NaN,사항\nAutolabeling / weak supervision / active l...,NaN,NaN,는 100% 지급\n커피챗\n지원 전 회사/팀/업무가 더 궁금하시면 커피챗(온·오프...,header+rule,0.733,False,moved_process_lines,주요업무:\n소개\n합류하면 풀게 될 문제\n비저너리의 데이터 자동 가공 파이프라인...
2,AI 에이전트,[로봇 솔루션기업] 모바일로봇 교통 관제 시스템 설계 및 개발자,(주)맨토스파워,서울 마포구,경력2년↑,대졸↑,정규직,~ 02/17(화),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52980063,...,NaN,"- 로봇, AI 관련 업무 유경험자\n- Multi-Agent Path Find...",접수기간 : 2026-02-02 10시 ~ 2026-02-17 24시\n제출서류 :...,NaN,NaN,header+rule,0.700,False,NaN,주요업무:\n[직무]\n모바일로봇 교통 관제 시스템 설계 및 개발자\n[\n]\n-...
3,AI 에이전트,Microsoft 365 Copilot 및 Agent 개발 컨설턴트,(주)무늬랩스,서울 송파구,경력2년↑,초대졸↑,정규직,~ 03/04(수),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52979929,...,"• 경력 2년 이상\n• 대학졸업(2,3년)이상\n• Copilot Agent 기획...","• Copilot Studio (Builder), Power Automate, Po...",서류전형 → 1차면접 (부문 리더) → 2차면접 (대표이사) → 최종합격\n면접일정...,및 혜택\n “일과 삶의 균형과 조화를 만들기 위해 노력합니다.” \n • 4대 보...,": 협의 후 결정 \n• 주근무지 : 서울 잠실 본사\n• 근무시간 : 주 5일, ...",header+rule,0.880,False,NaN,주요업무:\n고객 맞춤형 M365 Copilot & Agent 활용 방안 제안 및 ...
4,AI 에이전트,35. AI Platform Engineer [대기업 게임사],베스트네트워크㈜,서울 강남구,경력2년↑,대졸↑,정규직,~ 02/17(화),https://www.saramin.co.kr/zf_user/jobs/relay/v...,52979323,...,[필수요건]\n- 3년 이상 경력\n- 소프트웨어 엔지니어링: FE → BE → 인...,"]\n- LangChain, OpenAI API 등 LLM 활용 경험\n- Tool...",제출서류 : 사람인 온라인 이력서\n접수방법 : 사람인 입사지원\n전형절차 : 서류...,NaN,: 면접 후 결정 \n• 근무지 : 서울 강남구 언주로30길 13 \n 최호식 대표...,header+rule,0.806,False,moved_process_lines,주요업무:\n사내 유저들의 워크플로우를 분석·개선하여 공통 문제를 해결하고 획기적인...


In [17]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   keyword           478 non-null    str    
 1   title             478 non-null    str    
 2   company           478 non-null    str    
 3   location          478 non-null    str    
 4   career            478 non-null    str    
 5   education         478 non-null    str    
 6   work_type         478 non-null    str    
 7   deadline          478 non-null    str    
 8   link              478 non-null    str    
 9   rec_idx           478 non-null    int64  
 10  keyword_text      478 non-null    str    
 11  crawled_at        478 non-null    str    
 12  responsibilities  256 non-null    str    
 13  qualifications    291 non-null    str    
 14  preferred_quali   252 non-null    str    
 15  hiring_process    310 non-null    str    
 16  benefits          191 non-null    str    
 17  salary  